In [162]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import gc
from tqdm import tqdm
from numpy import random
from prophet import Prophet
from downcast import reduce
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from prophet_m5 import run_prophet
import multiprocessing  
from multiprocessing import Pool, cpu_count
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Importación de datos

## Historico de ventas

In [2]:
print('Cantidades vendidas de los ultimos 1913 días')
historico_ventas_df = pd.read_csv("./data/sales_train_validation.csv")
historico_ventas_df

Cantidades vendidas de los ultimos 1913 días


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [3]:
print('Cantidades vendidas de los ultimos 1941 días')
historico_ventas_eva_df = pd.read_csv("./data/sales_train_evaluation.csv")
historico_ventas_eva_df

Cantidades vendidas de los ultimos 1941 días


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


# Calendario

In [4]:
print('Calendario con multiple información')
calendar_df = pd.read_csv('./data/calendar.csv')
calendar_df

Calendario con multiple información


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


# Precio promedio

In [5]:
print('Precio de venta promedio por semana por artículo')
precio_promedio_df = pd.read_csv('./data/sell_prices.csv')
precio_promedio_df

Precio de venta promedio por semana por artículo


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


# Baseline

Usaremos una red LSTM sencilla como nuestra linea de partida base, este la mediremos con  RMSE y el resultado final \
con el clasificador ofrecido en la competencia.

In [5]:
d_cols = [c for c in historico_ventas_df.columns if 'd_' in c]
organize_df=historico_ventas_df[['id']+ d_cols].set_index('id').T
organize_df=organize_df.reset_index().rename(columns={'index':'d'})
organize_df=organize_df.iloc[:,1:]
d = dict.fromkeys(organize_df.select_dtypes(np.int64).columns, np.int16) #convert to int16 to consume less memory
organize_df = organize_df.astype(d)

Llevaremos los datos a un rango de 0 a 1 usando el MinMaxScaler, luego se organiza la data de entrenamiento con un tiempo de 14 para predecir el siguiente y finalmente se convierte a tipo float16.

In [48]:
x_train = organize_df.values

scaler = MinMaxScaler(feature_range=(0,1))
x_train_normalize = scaler.fit_transform(x_train)

timesteps=14
x_train_final = []
y_train_final = []

for i in range (timesteps, 1913):
    x_train_final.append(x_train_normalize[i-timesteps:i])
    y_train_final.append(x_train_normalize[i])
    
x_train_final = np.array(x_train_final,dtype='float16')
y_train_final = np.array(y_train_final,dtype='float16')

Se crea la red secuencial, se agregan las capas a utilizar y se entrena la red.

In [57]:
model = Sequential(name='first_try_total')
model.add(LSTM(units=100, return_sequences = True, input_shape=(x_train_final.shape[1],x_train_final.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units = 400))
model.add(Dense(y_train_final.shape[1],activation="linear",name='output_layer'))

model.compile(loss='mse',
              optimizer='adam')
history = model.fit(x_train_final, y_train_final, epochs=20, batch_size = 44)

Epoch 1/20
44/44 [==============================] - 23s 393ms/step - loss: 0.0171
Epoch 2/20
44/44 [==============================] - 17s 381ms/step - loss: 0.0133
Epoch 3/20
44/44 [==============================] - 17s 382ms/step - loss: 0.0130
Epoch 4/20
44/44 [==============================] - 17s 379ms/step - loss: 0.0127
Epoch 5/20
44/44 [==============================] - 16s 375ms/step - loss: 0.0123
Epoch 6/20
44/44 [==============================] - 17s 380ms/step - loss: 0.0122
Epoch 7/20
44/44 [==============================] - 17s 377ms/step - loss: 0.0120
Epoch 8/20
44/44 [==============================] - 17s 384ms/step - loss: 0.0118
Epoch 9/20
44/44 [==============================] - 17s 383ms/step - loss: 0.0117
Epoch 10/20
44/44 [==============================] - 17s 382ms/step - loss: 0.0117
Epoch 11/20
44/44 [==============================] - 17s 383ms/step - loss: 0.0115
Epoch 12/20
44/44 [==============================] - 17s 380ms/step - loss: 0.0115
Epoch 13/20
4

A continuación se hara una predicción de los próximos 28 días y posterior a esto se invertira la normalización utilizada previamente. Con esto podremos calcular el RMSE usando la data de validación.

In [58]:
x_test = x_train_normalize[-timesteps:]
predictions_accumulated = x_test[0].tolist()
for i in range(28):
    x_test = x_test[-14:]
    x_input = np.array([np.array(x_test)])
    predictions = model.predict(x_input)
    x_test = np.vstack((x_test,predictions))
    predictions_accumulated = np.vstack((predictions_accumulated,predictions.tolist()))
    
result_inverse =  scaler.inverse_transform(np.array(predictions_accumulated[-28:]))

y_test = historico_ventas_eva_df.iloc[:,-28:].values
print('EL valor de RMSe es de: ',np.sqrt(mean_squared_error(y_test,result_inverse.T)))

EL valor de RMSe es de:  2.2470986457997384


Se crea el file para adjuntar a la competencia y evaluar nuestro modelo.

In [15]:
cols_sub = ['id']
for i in range(1,29):
    cols_sub.append('F' + str(i))

In [59]:
cols = organize_df.columns
submission = pd.DataFrame(result_inverse,columns=cols).T.reset_index()
ventas_test = historico_ventas_eva_df.iloc[:, [0] + list(range(-28,0))]

ventas_test.columns = cols_sub
submission.columns = cols_sub
submission_final = submission.append(ventas_test)
submission_final.to_csv('./submissions/submission_lstm.csv',index=False)

#### El valor obtenido en la competencia es de: 1.17

In [62]:
del x_test, result_inverse, ventas_test, submission, submission_final, y_test, history, \
    x_train_final, x_train, x_train_normalize, timesteps, d, d_cols
gc.collect()
print('done')

done


# FB Prophet

Para el siguiente paso se creara un modelo sencillo de FB prophet donde solo se usara la data de ventas para predecir los 28 días siguientes. Para disminuir el tiempo de corrida se usara multiprocessing.

Nota: Recordar tener en la misma carpeta el archivo de python con el cual se correra prophet.

In [7]:
d_cols = [c for c in historico_ventas_df.columns if 'd_' in c]
organize_basic_df=historico_ventas_df[['id']+ d_cols].set_index('id').T
organize_basic_df=organize_basic_df.reset_index().rename(columns={'index':'d'})
organize_basic_df=pd.merge(organize_basic_df,calendar_df,how='left',left_on='d',right_on='d').set_index('date')
organize_basic_df=organize_basic_df.fillna('no_event')
organize_basic_df.to_csv('./data/organize_basic_df.csv')

In [9]:
%%time

products_id = organize_basic_df.iloc[:,1:-12].columns

if __name__ == '__main__':
    pool = Pool(cpu_count())
    predictions = pool.map(run_prophet, products_id)
    print('Done')

Done
Wall time: 4h 4min 58s


In [10]:
pool.terminate()

En los siguientes dos pasos sera como en el anterior, calcularemos el RMSE y luego subiremos los resultados para obtener
la calificacion oficial

In [13]:
submission = pd.DataFrame()
for i in range(len(predictions)):
    product_id = predictions[i][0]
    submission[product_id]=predictions[i][1]['yhat']

submission = submission.T.reset_index()
rmse_prophet = np.sqrt(mean_squared_error(historico_ventas_eva_df.iloc[:,-28:],submission.iloc[:,1:]))
print('EL valor de RMSe es de: ',rmse_prophet)

EL valor de RMSe es de:  2.3490222029951133


In [16]:
ventas_test = historico_ventas_eva_df.iloc[:, [0] + list(range(-28,0))]
ventas_test.columns = cols_sub
submission.columns = cols_sub
submission_final = submission.append(ventas_test)
submission_final.to_csv('./submissions/submission_prophet.csv',index=False)

#### El valor obtenido en la competencia es de: 0.82

## Usando regresores

La siguiente implementación tiene un alto consumo de RAM y toma bastante tiempo en realizar, tener en cuenta antes de seguir.

Changing the dataset help to run the model faster since it will only read the dataset and not merging everytime.

we will merge all 3 files to have more information available when running FB Prophet and get a better estimate.

In [31]:
%%time
calendar_df.rename(columns={"date": "ds"})
calendar_df['date'] = pd.to_datetime(calendar_df['date'],format='%Y-%m-%d')
calendar_df['DOY'] = calendar_df['date'].dt.dayofyear

Wall time: 12 ms


In [40]:
%%time
df_complete = pd.melt(historico_ventas_eva_df, id_vars=['id', 'item_id', 'dept_id',
    'cat_id', 'store_id', 'state_id'],var_name='d', value_name='y')
df_complete = pd.merge(df_complete, calendar_df, on='d', how='left')
df_complete = pd.merge(df_complete, precio_promedio_df, on=['store_id','item_id',
    'wm_yr_wk'], how='left')
df_complete = df_complete.fillna('0')

Wall time: 7min 21s


In [47]:
df_complete = reduce(df_complete)
desire_columns = ['id','state_id','date','y', 'wday', 'month', 'year', 'snap_CA','snap_TX','snap_WI','sell_price','DOY']
df_complete = df_complete[desire_columns]
df_complete['sell_price'] = df_complete['sell_price'].astype('float32')

In [59]:
df_complete.dtypes

id                  category
state_id            category
date          datetime64[ns]
y                      int16
wday                    int8
month                   int8
year                   int16
snap_CA                 int8
snap_TX                 int8
snap_WI                 int8
sell_price           float16
DOY                    int16
dtype: object

In [ ]:
del historico_ventas_df, calendar_df, precio_promedio_df

In [64]:
cat_complete = {'id':'category','state_id':'category','y':'int16','wday':'int8',
               'month':'int8','year':'int16','snap_CA':'int8','snap_TX':'int8','snap_WI':'int8',
                'sell_price':'float16','DOY':'int16'}

In [51]:
import warnings
warnings.filterwarnings('ignore')

In [73]:
predictions = pd.DataFrame()
for product_id in tqdm(df_complete['id'].unique()):
    estate = df_complete[df_complete['id']==product_id]['state_id'].unique()[0]
    snap = 'snap_' + estate
    desire_columns = ['date','y', 'wday', 'month', 'year', 'sell_price', snap, 'DOY']
    temp_dataset = df_complete[df_complete['id']==product_id]
    temp_dataset = temp_dataset[desire_columns]
    future_regressors = temp_dataset[temp_dataset['date']>="4/25/2016"]
    future_regressors = future_regressors.drop(['y'],axis=1)
    future_regressors.columns = ['ds','add_1','add_2','add_3','add_4','add_5','add_6']
    temp_dataset = temp_dataset[temp_dataset['date']<"4/25/2016"]
    temp_dataset = temp_dataset[desire_columns]
    temp_dataset.columns = ['ds','y','add_1','add_2','add_3','add_4','add_5','add_6']
    temp_dataset['ds'] = temp_dataset['ds'].astype('string')
    
    #define prophet params
    model = Prophet(
        growth='linear',
        daily_seasonality = True,
        weekly_seasonality = True,
        yearly_seasonality = True
    )
    model.add_country_holidays(country_name='US')
    model.add_seasonality(name='monthly', period=30.5, fourier_order=12)
    model.add_regressor('add_1')
    model.add_regressor('add_2')
    model.add_regressor('add_3')
    model.add_regressor('add_4')
    model.add_regressor('add_5')
    model.add_regressor('add_6')
    
    #train model
    model.fit(temp_dataset)

    #predict future values
    future = model.make_future_dataframe(periods=28,
                                        include_history=False,
                                        freq='1D')
    future.append(future_regressors)
    future = pd.merge(future,future_regressors)
    predictions[product_id] = forecast['yhat']

100%|█████████████████████████████████████████████████████████████████████████| 30490/30490 [33:23:51<00:00,  3.94s/it]


In [160]:
submission = predictions.T.reset_index()
rmse_prophet = np.sqrt(mean_squared_error(historico_ventas_eva_df.iloc[:,-28:],submission.iloc[:,1:]))
print('EL valor de RMSe es de: ',rmse_prophet)

EL valor de RMSe es de:  2.017868731079835


In [161]:
ventas_test = historico_ventas_eva_df.iloc[:, [0] + list(range(-56,-28))]
submission = submission.iloc[:,[0] + list(range(-28,0))]
submission['index'] = submission['index'].str.replace('evaluation','validation')
ventas_test.columns = cols_sub
submission.columns = cols_sub
submission_final = submission.append(ventas_test)
submission_final.to_csv('./submissions/submission_prophet_regressors.csv',index=False)

#### El valor obtenido en la competencia es de: 0.79

# LightBGM

En este caso no aplicaremos como tal la ténica, pero en la competencia quedo demostrado que para este problema en particular LightGBM ofrece el mejor desempeño de todos los modelos. Tiene un tiempo corto de entrenamiento con una alta precision.

En este caso, a través de la web el notebook mas destacado es el del usuario kkiller de kaggle. Aunque los parametros fueron esenciales a la hora del entrenamiento, lo mas importante a destacar fue el saber elegir los atributos a utilizar. El trabajo de feature engineer fue realmente increible y algo de lo que aprender.

A continuación dejo el link para que se pueda estudiar a manera más profunda cada paso que se dio.

https://www.kaggle.com/kneroma/m5-first-public-notebook-under-0-50

# Conclusion

En la actualidad el trabajo de series de tiempo no es tan enfocado a realizar los algoritmos matematicos clasicos para realizar la serie, debido a la gran cantidad de herramientas que abundan en la web. El analisis de los datos y la correcta extracion de atributos claves se ha convertido en un punto clave para esta area.

Además, no se debe pensar que solo los modelos de redes artificiales son adecuados, los modelos estadisticos clasicos mediante Potenciación del gradiente en sistemas de arboles son muy eficientes y rapidos de correr. Cada dia la facilidad y exactitud de las series de tiempo mejoran, volviendose mas amigables con los usuarios.